In [5]:
import sys
sys.path.insert(0, '../../')

import graph
import importlib
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import schedule as sch
import topo
import typing
import utils
import yaml
from tqdm.notebook import tqdm

matplotlib.rc('font', family='Times New Roman', size=16)

In [6]:
def unit_size_cb(r: int):
    return 10000 * math.pow(10, random.randint(0, 1))

def gen_graphs(graph_count, source_selector_dict, max_source_num):
    source_selector = graph.MultiDomainSourceSelector(source_selector_dict)
    global source_quota
    source_quota = max_source_num
    def source_num(i):
        global source_quota
        assert graph_count - i <= source_quota
        delta = source_quota - (graph_count - i)
        ret = 1 + random.randint(0, min(delta, 2))
        source_quota -= ret
        return ret
    gen_args_list = [
            {
            "total_rank": random.randint(3, 7),
            "max_node_per_rank": random.randint(1, 3),
            "max_predecessors": random.randint(1, 2),
            "mi_cb": lambda: 1,
            "memory_cb": lambda: int(2e8),
            "unit_size_cb": unit_size_cb,
            "unit_rate_cb": lambda: random.randint(10, 20),
            "source_hosts": source_selector,
            "sink_hosts": ["cloud1"],
            "sources_num": source_num(i),
        }
        for i in range(graph_count)
    ]
    return [
        graph.MultiSourceGraphGenerator("g" + str(idx), **gen_args).gen_dag_graph()
        for idx, gen_args in enumerate(gen_args_list)
    ]

In [7]:
from tqdm.notebook import trange

def big_avg(array):
    return sum(array) / len(array)

def run(sc, labels, source_selector_dict, max_source_num):
    flow_bp_data = [[] for _ in labels]
    all_cloud_bp_data = [[] for _ in labels]
    sbon_bp_data = [[] for _ in labels]
    flow_event_data = [[] for _ in labels]
    all_cloud_event_data = [[] for _ in labels]
    sbon_event_data = [[] for _ in labels]
    edge_random_event_data = [[] for _ in labels]
    test_round = 100
    for idx, graph_count in enumerate(labels):
        print(graph_count)
        for _ in trange(test_round, desc="running with {} graphs".format(graph_count)):
            graph_list = gen_graphs(graph_count, source_selector_dict, max_source_num)
            sc.topo.clear_occupied()
            flow_scheduler = sch.MultiEdgeFlowScheduler(sc)
            flow_calculator = sch.LatencyCalculator(sc.topo)
            flow_result_list = flow_scheduler.schedule_multiple(graph_list)
            for g, result in zip(graph_list, flow_result_list):
                assert result is not None
                flow_calculator.add_scheduled_graph(g, result)
            flow_latency, flow_bp, _ = flow_calculator.compute_latency()
            for g in graph_list:
                flow_event_data[idx].append(flow_latency[g.uuid] / g.number_of_vertices())
                flow_bp_data[idx].append(flow_bp[g.uuid])

            sc.topo.clear_occupied()
            sbon_scheduler = sch.MultiEdgeSbonScheduler(sc)
            sbon_calculator = sch.LatencyCalculator(sc.topo)
            sbon_result_list = sbon_scheduler.schedule_multiple(graph_list)
            for g, result in zip(graph_list, sbon_result_list):
                assert result is not None
                sbon_calculator.add_scheduled_graph(g, result)
            sbon_latency, sbon_bp, _ = sbon_calculator.compute_latency()
            for g in graph_list:
                sbon_event_data[idx].append(sbon_latency[g.uuid] / g.number_of_vertices())
                sbon_bp_data[idx].append(sbon_bp[g.uuid])

            sc.topo.clear_occupied()
            all_cloud_scheduler = sch.MultiEdgeAllCloudScheduler(sc)
            all_cloud_calculator = sch.LatencyCalculator(sc.topo)
            all_cloud_result_list = all_cloud_scheduler.schedule_multiple(graph_list)
            for g, result in zip(graph_list, all_cloud_result_list):
                assert result is not None
                all_cloud_calculator.add_scheduled_graph(g, result)
            all_cloud_latency, all_cloud_bp, _ = all_cloud_calculator.compute_latency()
            for g in graph_list:
                all_cloud_event_data[idx].append(all_cloud_latency[g.uuid] / g.number_of_vertices())
                all_cloud_bp_data[idx].append(all_cloud_bp[g.uuid])
            # all_cloud_event_data[idx].extend(list(all_cloud_latency.values()))

            # sc.topo.clear_occupied()
            # edge_random_scheduler = sch.EdgeRandomScheduler(sc)
            # edge_random_calculator = sch.LatencyCalculator(sc.topo)
            # edge_random_result_list = edge_random_scheduler.schedule_multiple(graph_list)
            # for g, result in zip(graph_list, edge_random_result_list):
            #     assert result is not None
            #     edge_random_calculator.add_scheduled_graph(g, result)
            # edge_random_latency, edge_random_bp = edge_random_calculator.compute_latency()
            # edge_random_event_data[idx].extend(list(edge_random_latency.values()))

            # for k in all_cloud_latency.keys():
                # flow_event_data[idx].append(flow_latency[k] / all_cloud_latency[k])
                # edge_random_event_data[idx].append(edge_random_latency[k] / flow_latency[k])
    flow_event_data = [big_avg(i) for i in flow_event_data]
    flow_bp_data = [big_avg(i) for i in flow_bp_data]
    all_cloud_event_data = [big_avg(i) for i in all_cloud_event_data]
    all_cloud_bp_data = [big_avg(i) for i in all_cloud_bp_data]
    sbon_event_data = [big_avg(i) for i in sbon_event_data]
    sbon_bp_data = [big_avg(i) for i in sbon_bp_data]
    # return flow_event_data, flow_bp_data, all_cloud_event_data, all_cloud_bp_data, sbon_event_data, sbon_bp_data
    return flow_event_data, flow_bp_data, all_cloud_event_data, all_cloud_bp_data, sbon_event_data, sbon_bp_data
    # ax_lat.plot(flow_event_data)
    # ax_lat.plot(all_cloud_event_data)
    # ax_bp.plot(flow_bp_data)
    # ax_bp.plot(all_cloud_bp_data)
    # plt.eventplot(flow_event_data, lineoffsets=[i-width for i in range(len(labels))], linelengths=width, linewidths=0.5, colors="C0", orientation="vertical")
    # plt.eventplot(edge_random_event_data, lineoffsets=[i for i in range(len(labels))], linelengths=width, linewidths=0.5, colors="C1", orientation="vertical")
    # plt.eventplot(all_cloud_event_data, lineoffsets=[i+width for i in range(len(labels))], linelengths=width, linewidths=0.5, colors="C2", orientation="vertical")
    # plt.violinplot(flow_event_data, vert=True, positions=[i-width for i in range(len(labels))], widths=width)
    # plt.violinplot(edge_random_event_data, vert=True, positions=[i for i in range(len(labels))], widths=width)
    # plt.violinplot(all_cloud_event_data, vert=True, positions=[i+width for i in range(len(labels))], widths=width)

In [8]:
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e6h.yaml", "r").read(), Loader=yaml.Loader))
f_lat1, f_bp1, ac_lat1, ac_bp1, sbon_lat1, sbon_bp1 = run(sc, [4, 8, 12, 16, 20], 
                    {"edge0": {'e0rasp1': 10, 'e0rasp2': 10, 'e0rasp3': 10},
                    "edge1": {'e1rasp1': 10, 'e1rasp2': 10, 'e1rasp3': 10}}, 45)
print(f_lat1)
print(ac_lat1)
print(sbon_lat1)

4


running with 4 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

8


running with 8 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

12


...

16


running with 16 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

20


running with 20 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

[4.9554497774557555, 6.253373150989393, 7.139688629184887, 8.494548739715777, 9.317850308322964]
[5.20076692350515, 6.667410312556477, 7.8457179014663065, 9.366189257634916, 10.482070555838819]
[6.101054481086619, 8.132374581244862, 9.593855000005806, 11.448456410023663, 12.634542008030117]


In [9]:
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e18h.yaml", "r").read(), Loader=yaml.Loader))
edge0 = {"e0rasp"+str(i): 12 for i in range(1, 7)}
edge1 = {"e1rasp"+str(i): 12 for i in range(1, 4)}
edge1.update({"e1vm"+str(i): 24 for i in range(1, 4)})
edge2 = {"e2rasp"+str(i): 12 for i in range(1, 5)}
edge2.update({"e2vm"+str(i): 24 for i in range(1, 3)})
source_dict = {"edge0": edge0, "edge1": edge1, "edge2": edge2}
f_lat2, f_bp2, ac_lat2, ac_bp2, sbon_lat2, sbon_bp2 = run(sc, [40, 60, 80, 100, 120], source_dict, 200)
print(f_lat2)
print(ac_lat2)
print(sbon_lat2)

40


running with 40 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

60


running with 60 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

80


running with 80 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
sc = topo.Scenario.from_dict(yaml.load(open("../../samples/1e54h.yaml", "r").read(), Loader=yaml.Loader))
edge0 = {"e0rasp"+str(i): 12 for i in range(1, 9)}
edge0.update({"e0vm"+str(i): 24 for i in range(1, 7)})
edge0.update({"e0desktop"+str(i): 48 for i in range(1, 5)})
edge1 = {"e1rasp"+str(i): 12 for i in range(1, 9)}
edge1.update({"e1vm"+str(i): 24 for i in range(1, 7)})
edge1.update({"e1desktop"+str(i): 48 for i in range(1, 5)})
edge2 = {"e2rasp"+str(i): 12 for i in range(1, 9)}
edge2.update({"e2vm"+str(i): 24 for i in range(1, 7)})
edge2.update({"e2desktop"+str(i): 48 for i in range(1, 5)})
source_dict = {"edge0": edge0, "edge1": edge1, "edge2": edge2}
f_lat3, f_bp3, ac_lat3, ac_bp3, sbon_lat3, sbon_bp3 = run(sc, [200, 300, 400, 500, 600], source_dict, 1000)
print(f_lat3, ac_lat3)

200


running with 200 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

300


running with 300 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

400


...

500


running with 500 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

600


running with 600 graphs:   0%|          | 0/100 [00:00<?, ?it/s]

[20.600970834214586, 28.825593371594866, 38.019014994523296, 46.47901277889612, 54.31786919027147] [24.360759401022886, 34.35613662975439, 44.858573265063654, 55.08710536934305, 64.83622267279185]


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5), dpi=300, constrained_layout=True)
fig.supxlabel("Number of Graphs")
fig.supylabel("Average Latency per Operator (ms)")
axes[0].set_xticks([i for i in range(5)])
axes[0].set_xticklabels([4, 8, 12, 16, 20])
axes[1].set_xticks([i for i in range(5)])
axes[1].set_xticklabels([40, 60, 80, 100, 120])
axes[2].set_xticks([i for i in range(5)])
axes[2].set_xticklabels([200, 300, 400, 500, 600])
axes[0].plot(f_lat1, marker="s")
axes[0].plot(ac_lat1, marker="^")
axes[0].plot(sbon_lat1, marker="o")
# axes[1].plot(f_lat2, marker="s")
# axes[1].plot(ac_lat2, marker="^")
# axes[1].plot(sbon_lat2, marker="o")
# axes[2].plot(f_lat3, marker="s")
# axes[2].plot(ac_lat3, marker="^")
plt.show()